In [2]:
from collections import Counter
import os
import random
import re
import time

In [3]:
from bs4 import BeautifulSoup
import cloudscraper
import geopandas as gpd
import pandas as pd

In [4]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [5]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [6]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [7]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [8]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [9]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Five Below Data

In [12]:
def get_coords(url: str) -> Point:
    r = scraper.get(url)
    if r.status_code != 200:
        print("Status Code:", r.status_code, url)
        return {}
    try:
        soup = BeautifulSoup(r.text, "html.parser")
        lat = float(soup.find("meta", {"itemprop": "latitude"}).attrs["content"])
        lon = float(soup.find("meta", {"itemprop": "longitude"}).attrs["content"])
    except Exception as e:
        print(e, url)
    return Point(lon, lat)

In [11]:
scraper = cloudscraper.create_scraper()

In [ ]:
five_below_list = []

root_url = "https://locations.fivebelow.com/"
states_url = os.path.join(root_url, "directory")
r_all_states = scraper.get(states_url)
soup = BeautifulSoup(r_all_states.text, "html.parser")
state_as = soup.find_all("a", {"class": "Directory-listLink"})

for state_a in tqdm(state_as, desc="Parsing States"):
    state_code = state_a.attrs["href"].split("/")[0]
    state_store_list = []

    # If File exists continue
    if os.path.isfile(f"data/states/{state_code}.gpkg"):
        continue

    state_url = os.path.join(root_url, state_code)

    r_state = scraper.get(state_url)
    soup_state = BeautifulSoup(r_state.text, "html.parser")
    location_as = soup_state.find_all("a", {"class": "Directory-listLink"})

    for location_a in tqdm(location_as, desc=f"Parsing Locations in {state_code}"):
        location_href = "/".join(location_a.attrs["href"].split("/")[0:2])
        location_url = os.path.join(root_url, location_href)
        r_location = scraper.get(location_url)
        soup_location = BeautifulSoup(r_location.text, "html.parser")
        store_as = soup_location.find_all("a", {"class": "Teaser-titleLink"})

        for store_a in store_as:
            store_url = store_a.attrs["href"].replace("../", root_url)
            point = get_coords(store_url)
            store_dict = {"STATE": state_code.upper(), "geometry": point}
            five_below_list.append(store_dict)
            state_store_list.append(store_dict)

        time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        five_below_state_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        five_below_state_gdf.to_file(f"data/states/{state_code}.gpkg")

Parsing States:   0%|          | 0/45 [00:00<?, ?it/s]

Parsing Locations in al:   0%|          | 0/29 [00:00<?, ?it/s]

Parsing Locations in az:   0%|          | 0/20 [00:00<?, ?it/s]

Parsing Locations in ar:   0%|          | 0/11 [00:00<?, ?it/s]

Parsing Locations in ca:   0%|          | 0/122 [00:00<?, ?it/s]

...

Parsing Locations in ma:   0%|          | 0/35 [00:00<?, ?it/s]

Parsing Locations in mi:   0%|          | 0/49 [00:00<?, ?it/s]

Parsing Locations in mn:   0%|          | 0/25 [00:00<?, ?it/s]

Parsing Locations in ms:   0%|          | 0/18 [00:00<?, ?it/s]

Parsing Locations in mo:   0%|          | 0/35 [00:00<?, ?it/s]

In [62]:
five_below_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        five_below_gdf = gpd.read_file(constructed_path)
        five_below_gdfs.append(five_below_gdf)

In [ ]:
five_below_gdf = gpd.GeoDataFrame(pd.concat(five_below_gdfs, ignore_index=True))

In [ ]:
five_below_gdf = five_below_gdf.to_crs(9311)
five_below_gdf.to_file("data/five_belows.gpkg")

In [80]:
five_below_counts_gdf = pd.DataFrame(
    five_below_gdf.groupby("STATE").size(), columns=["FIVE_BELOW"]
)
five_below_counts_gdf = five_below_counts_gdf.reset_index()
five_below_counts_gdf = five_below_counts_gdf.rename(columns={"STATE": "STUSPS"})

## Combine With States

In [88]:
five_below_by_states_gdf = states_with_population_df.merge(
    five_below_counts_gdf, on="STUSPS", how="left"
)

In [90]:
five_below_by_states_gdf = five_below_by_states_gdf.fillna(0)
five_below_by_states_gdf["FIVE_BELOW"] = five_below_by_states_gdf["FIVE_BELOW"].astype(
    int
)

In [95]:
five_below_by_states_gdf["per_1000"] = five_below_by_states_gdf["FIVE_BELOW"] / (
    five_below_by_states_gdf["POPULATION"] / 1000
)
five_below_by_states_gdf["per_10k"] = five_below_by_states_gdf["FIVE_BELOW"] / (
    five_below_by_states_gdf["POPULATION"] / 10_000
)
five_below_by_states_gdf["per_100k"] = five_below_by_states_gdf["FIVE_BELOW"] / (
    five_below_by_states_gdf["POPULATION"] / 100000
)
five_below_by_states_gdf["per_500k"] = five_below_by_states_gdf["FIVE_BELOW"] / (
    five_below_by_states_gdf["POPULATION"] / 500_000
)
five_below_by_states_gdf["per_1m"] = five_below_by_states_gdf["FIVE_BELOW"] / (
    five_below_by_states_gdf["POPULATION"] / 1_000_000
)

In [96]:
five_below_by_states_gdf = five_below_by_states_gdf.dropna()

In [97]:
five_below_by_states_gdf = five_below_by_states_gdf.to_crs(9311)
five_below_by_states_gdf.to_file("data/five_below_per_state.gpkg")